# CAFA6-02-Enhanced feature extraction

Improvement from CAFA6-01: 71 features --> 77 features.

6 new features come from:
- Add secondary structure predictions (Helix/Sheet/Coil fractions).
- Include global physicochemical descriptors (pI, GRAVY, instability index).

References:
- https://www.kaggle.com/code/analyticaobscura/cafa-6-decoding-protein-mysteries

---

In [1]:
!pip install biopython > /dev/null

In [2]:
from Bio import SeqIO  # parse fasta file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# file paths
TRAIN_TERMS = "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv"
TRAIN_SEQ = "/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta"
TRAIN_TAXONOMY = "/kaggle/input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv"
TEST_SEQ = "/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta"
GO_WEIGHTS = "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv"

## Step 1: Turn into a unified dataFrame

---

In [4]:
train_terms_df = pd.read_csv(TRAIN_TERMS, sep="\t")  # identifier --> label
train_taxonomy_df = pd.read_csv(TRAIN_TAXONOMY, sep="\t", names=['EntryID', 'taxonomyID'])  # identifier --> taxonomy
ia_df = pd.read_csv(GO_WEIGHTS, sep='\t', names=['term', 'ia'])  # GO --> weight

def load_fasta_to_dataframe(file_path, is_train=True):
    records = []
    parser = SeqIO.parse(file_path, "fasta")
    for record in parser:
        entry_id = record.id.split('|')[1] if is_train and '|' in record.id else record.id.split()[0]
        records.append({'EntryID': entry_id, 'sequence': str(record.seq)})
    return pd.DataFrame(records)

train_sequences_df = load_fasta_to_dataframe(TRAIN_SEQ, is_train=True)  # identifier --> input: amino acid sequence

In [5]:
print(train_terms_df[0:2], "\n---")
print(train_taxonomy_df[0:2], "\n---")
print(ia_df[0:2], "\n---")
print(train_sequences_df[0:2], "\n---")

  EntryID        term aspect
0  Q5W0B1  GO:0000785      C
1  Q5W0B1  GO:0004842      F 
---
      EntryID  taxonomyID
0  A0A0C5B5G6        9606
1      A0JNW5        9606 
---
         term        ia
0  GO:0000001  0.000000
1  GO:0000002  2.849666 
---
      EntryID                                           sequence
0  A0A0C5B5G6                                   MRWQEMGYIFYPRKLR
1      A0JNW5  MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL... 
---


In [6]:
# Group the info above into a single data frame
protein_labels = train_terms_df.groupby('EntryID')['term'].apply(list).reset_index(name='labels')  # turn all EntryID duplicates into one EntryID with their terms forming a list
train_df_eda = pd.merge(train_sequences_df, train_taxonomy_df, on='EntryID', how='left')
train_df_eda = pd.merge(train_df_eda, protein_labels, on='EntryID', how='inner')
train_df_eda['seq_length'] = train_df_eda['sequence'].str.len()
train_df_eda['num_labels'] = train_df_eda['labels'].str.len()

In [7]:
print(protein_labels[82399:82401])

      EntryID                    labels
82399  X2JI34  [GO:0106223, GO:0051762]
82400  X4Y2L4  [GO:0033906, GO:0030214]


In [8]:
# Unified data frame
train_df_eda

,EntryID,sequence,taxonomyID,labels,seq_length,num_labels
0,A0A0C5B5G6,MRWQEMGYIFYPRKLR,9606,"[GO:0001649, GO:0033687, GO:0005615, GO:000563...",16,14
1,A0JNW5,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,9606,"[GO:0120013, GO:0034498, GO:0005769, GO:012000...",1464,8
2,A0JP26,MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...,9606,[GO:0005515],581,1
3,A0PK11,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,9606,"[GO:0007605, GO:0005515]",232,2
4,A1A4S6,MGLQPLEFSDCYLDSPWFRERIRAHEAELERTNKFIKELIKDGKNL...,9606,"[GO:0005829, GO:0010008, GO:0005515, GO:000509...",786,5
...,...,...,...,...,...,...
82399,Q9UTM1,MSKLKAQSALQKLIESQKNPNANEDGYFRRKRLAKKERPFEPKKLV...,284812,[GO:0005730],112,1
82400,Q9Y7I1,MSSNSNTDHSTGDNRSKSEKQTDLRNALRETESHGMPPLRGPAGFP...,284812,"[GO:0005634, GO:0005829]",78,2
82401,Q9Y7P7,MRSNNSSLVHCCWVSPPSLTRLPAFPSPRILSPCYCYNKRIRPFRG...,284812,"[GO:0005634, GO:0005829]",117,2
82402,Q9Y7Q3,MHSSRRKYNDMWTARLLIRSDQKEEKYPSFKKNAGKAINAHLIPKL...,284812,"[GO:0005634, GO:0005739, GO:0005829]",149,3


## Step 2: Exploratory Data Analysis (EDA)

Already implemented in a local notebook

---

EDA - Summary
- `82,404` training examples (each example is a sequence of amino acids of different lengths)
- The `20/1381` most frequent taxonomyIDs (i.e. species) take up `91%` of the dataset 
- The `700/2906` most probable sequence lengths are within `[10,926]` which take up `81%` of the dataset. The most frequent sequence length is `359` characters.
- There is a weak positive correlation between `seq_length` and `num_labels`, meaning longer sequences sometimes yield more GOs
- Some `taxonomyIDs` have more `num_labels` than others (i.e. the mean `num_labels` is `2.791703`, while 25% of the  data has more than 3.375 and up to 16 `num_labels`)
- The annotation:
  - `537,027` annotated GO terms
  - `26,125` unique GO terms
  - Out of the three sub-ontologies, `Biological Process (P)` is the most annotated, then `Cellular Component (C)`, finally `Molecular Function (F)`
- (💡) A standard protein sequence is made up of 20 amino acid letters, each represented by a single uppercase English letter. 
  - A, C, D, E, F, G, H, I, K, L, M, N, P, Q, R, S, T, V, W, Y
  - Besides the 20 standard amino acids, protein sequences sometimes include:
    - `B` --> Aspartic acid (D) or Asparagine (N)
    - `Z` --> Glutamic acid (E) or Glutamine (Q)
    - `X` --> Unknown amino acid
    - `J` --> Leucine (L) or Isoleucine (I)
    - `U` --> Selenocysteine
    - `O` --> Pyrrolysine

## Step 3: Feature extraction

For the baseline performance, we only use the column `sequence` in our unified dataFrame to extract the features.

Here are some methods to turn the sequence into meaningful numerical values
- Amino Acid Composition (AAC)
  - Computes the relative frequency of each of the 20 standard amino acids.  
  - Captures the overall chemical makeup of the protein.
- Physicochemical Properties
  - Global properties correlated with structure and function, including:  
    - Hydrophobic fraction  
    - Charged amino acid fraction  
    - Molecular weight (log-scaled)  
    - Sequence length (log-scaled)
- CTD-style Group Composition 
  - Groups amino acids by shared chemical characteristics and measures their proportions, such as:  
    - Hydrophobic  
    - Polar  
    - Positive charge  
    - Negative charge  
    - Aromatic  
    - Aliphatic  
    - Small residues
- k-mer Frequencies (dipeptides and tripeptides)
  - Counts the frequency of selected short patterns (2-mers and 3-mers).  
  - Captures local sequence motifs that composition alone cannot represent.
- Secondary Structure Fractions
  - Estimates the proportion of each type of secondary structure in the protein:  
    - Alpha-helix  
    - Beta-sheet  
    - Coil  
  - Provides structural context that affects protein function.
- Global Physicochemical Descriptors
  - Includes sequence-level chemical properties:  
    - Isoelectric point (pI)  
    - Hydropathy index (GRAVY)  
    - Instability index  
  - Helps capture overall biochemical characteristics affecting stability and interactions.
  
---

In [9]:
AA_ORDER = 'ACDEFGHIKLMNPQRSTVWY'

AA_WEIGHTS = {
    'A': 89, 'C': 121, 'D': 133, 'E': 147, 'F': 165, 'G': 75, 'H': 155,
    'I': 131, 'K': 146, 'L': 131, 'M': 149, 'N': 132, 'P': 115, 'Q': 146,
    'R': 174, 'S': 105, 'T': 119, 'V': 117, 'W': 204, 'Y': 181
}

CTD_GROUPS = {
    'hydrophobic': set('AILMFWYV'),
    'polar': set('STNQ'),
    'positive': set('RK'),
    'negative': set('DE'),
    'aromatic': set('FWY'),
    'aliphatic': set('ILV'),
    'small': set('ACDGNPSTV'),
}

TOP_DIPEPTIDES = [
    'AL','LA','LE','EA','AA','AS','SA','EL','LL','AE',
    'SE','ES','GA','AG','VA','AV','LV','VL','LS','SL'
]

TOP_TRIPEPTIDES = [
    'ALA','LEA','EAL','LAL','AAA','LLE','ELE','ALE','GAL','ASA',
    'VLA','LAV','SLS','LSL','GLA','LAG','AVL','VLA','SLE','LES'
]

In [10]:
from collections import Counter
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import numpy as np

def extract_sequence_features(seq):
    """Extracts 77 numerical features from a protein sequence, safely ignoring unknown amino acids."""
    if not seq:
        return np.zeros(77)

    length = len(seq)
    aa_counts = Counter(seq)

    # --- 1. AAC (20 features)
    AA_ORDER = 'ACDEFGHIKLMNPQRSTVWY'
    aa_freq = np.array([aa_counts.get(aa, 0) / length for aa in AA_ORDER])

    # --- 2. Physicochemical (4 features)
    AA_WEIGHTS = {'A': 89, 'C': 121, 'D': 133, 'E': 147, 'F': 165, 'G': 75,
                  'H': 155, 'I': 131, 'K': 146, 'L': 131, 'M': 149, 'N': 132,
                  'P': 115, 'Q': 146, 'R': 174, 'S': 105, 'T': 119, 'V': 117,
                  'W': 204, 'Y': 181}
    hydrophobic = sum(aa_counts.get(a, 0) for a in 'AILMFWYV') / length
    charged = sum(aa_counts.get(a, 0) for a in 'DEKR') / length
    mol_weight = sum(aa_counts.get(aa, 0) * AA_WEIGHTS.get(aa, 0) for aa in aa_counts if aa in AA_WEIGHTS)
    physchem = np.array([np.log1p(length), hydrophobic, charged, np.log1p(mol_weight)])

    # --- 3. CTD group composition (7 features)
    CTD_GROUPS = {
        'hydrophobic': set('AILMFWYV'),
        'polar': set('STNQ'),
        'positive': set('RK'),
        'negative': set('DE'),
        'aromatic': set('FWY'),
        'aliphatic': set('ILV'),
        'small': set('ACDGNPSTV')
    }
    ctd = np.array([sum(1 for aa in seq if aa in group) / length for group in CTD_GROUPS.values()])

    # --- 4. Dipeptides (20 features)
    TOP_DIPEPTIDES = ['AL', 'LA', 'LE', 'EA', 'AA', 'AS', 'SA', 'EL', 'LL', 'AE',
                      'SE', 'ES', 'GA', 'AG', 'VA', 'AV', 'LV', 'VL', 'LS', 'SL']
    if length > 1:
        dip_counts = Counter(seq[i:i+2] for i in range(length - 1))
        dipep = np.array([dip_counts.get(dp, 0) / (length - 1) for dp in TOP_DIPEPTIDES])
    else:
        dipep = np.zeros(len(TOP_DIPEPTIDES))

    # --- 5. Tripeptides (20 features)
    TOP_TRIPEPTIDES = ['ALA', 'LEA', 'EAL', 'LAL', 'AAA', 'LLE', 'ELE', 'ALE', 'GAL', 'ASA',
                       'VLA', 'LAV', 'SLS', 'LSL', 'GLA', 'LAG', 'AVL', 'VLA', 'SLE', 'LES']
    if length > 2:
        tri_counts = Counter(seq[i:i+3] for i in range(length - 2))
        tripep = np.array([tri_counts.get(tp, 0) / (length - 2) for tp in TOP_TRIPEPTIDES])
    else:
        tripep = np.zeros(len(TOP_TRIPEPTIDES))

    # --- 6. Secondary structure + global descriptors (6 features)
    safe_seq = ''.join([aa for aa in seq if aa in AA_ORDER])
    if safe_seq:
        analysed_seq = ProteinAnalysis(safe_seq)
        # Secondary structure fractions
        helix, sheet, coil = analysed_seq.secondary_structure_fraction()
        ss_fraction = np.array([helix, sheet, coil])
        # Global descriptors: pI, GRAVY, instability index
        global_physchem = np.array([
            analysed_seq.isoelectric_point(),
            analysed_seq.gravy(),
            analysed_seq.instability_index(),
        ])
    else:
        ss_fraction = np.zeros(3)
        global_physchem = np.zeros(3)

    # concatenate all 77 features
    return np.concatenate([aa_freq, physchem, ctd, dipep, tripep, ss_fraction, global_physchem])

In [11]:
# Extract features for training. This will be the same as the AAC matrix above.
# However, in this code, we also create y_train_proteins which contains the EntryID of all 82404 examples. This will be used later for label encoding.

from tqdm import tqdm

features, protein_identifiers = list(), list()

entry_ids = set(train_terms_df['EntryID'].unique())
train_sequences_dict = train_sequences_df.set_index('EntryID')['sequence'].to_dict()  # simply turn train_sequences_df into a dict

for entry_id, sequence in tqdm(train_sequences_dict.items(), desc="Processing Train Sequences"):
    if entry_id in entry_ids:
        features.append(extract_sequence_features(sequence))
        protein_identifiers.append(entry_id)
        
X_train = np.array(features)
print(f"Training feature matrix shape: {X_train.shape}")

Processing Train Sequences: 100%|██████████| 82404/82404 [01:08<00:00, 1205.49it/s]


Training feature matrix shape: (82404, 77)


In [12]:
print(len(protein_identifiers))

82404


💡 As illustrated above, the features `X_train` are of shape (82404, 77) and the `protein_identifies` stores 82404 entryIDs accordingly.

## Step 4: Label encoding and Training

I have tried two methods of encoding the labels, but only the latter works.

- Method 1: Turning 26125 unique GO terms into multi-hot vectors
    - This results in `y_train` of shape `(82404, 26125)`, which is too large, extremely sparse, and makes the model’s final layer enormous.
    - This method fails
- Method 2: We divide the label in method 1 into three subsets (one for MF, one for BP, and one for CC).
    -  As a result, for each sequence, we will have 3 vectors as follows using multi-hot encoding (i.e. simply one-hot for multi-classification problems)
        - For MF: [0 1 0 1 ... 0] of length num_unique(MF_GO_temrs)
        - For BP: [0 1 0 1 ... 0] of length num_unique(BP_GO_temrs)
        - For CC: [0 1 0 1 ... 0] of length num_unique(CC_GO_temrs)
    - Then we will train three separate models for each ontology. We use three models to predict for a single example in the test set, and gather the predictions.

---

In [13]:
# Create three label sets
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

mlb_dict = dict()
models = dict()

for aspect in tqdm(['P', 'C', 'F'], desc="Training models"):
    # Filter the train_terms_df based on aspect
    ont_terms_df = train_terms_df[train_terms_df['aspect'] == aspect]

    # Group the dataFrame based on the EntryID, turn all the GO terms to a list, finally turns this dataFrame to a dict {entryID: terms}
    protein_terms = ont_terms_df.groupby('EntryID')['term'].apply(list).to_dict()

    # Create a list of labels for this aspect, if an entryID doesn't exist in this aspect, give it a []
    # This ensures y_train is of shape (82404, ...)
    labels = [protein_terms.get(entry_id, []) for entry_id in protein_identifiers]

    # Multi-hot encoding, use sparse representation
    mlb = MultiLabelBinarizer(sparse_output=True)
    y_train = mlb.fit_transform(labels)
    
    print(f"y_train shape for {aspect} ontology: {y_train.shape} \t\t Number of unique {aspect} terms: {y_train.shape[1]}")

    # Save to dict
    mlb_dict[aspect] = mlb
    model = OneVsRestClassifier(LogisticRegression(max_iter=600, solver='lbfgs', C=0.5, random_state=42), n_jobs=-1)
    model.fit(X_train, y_train)
    models[aspect] = model
    print(f"Model for {aspect} trained successfully.")

Training models:   0%|          | 0/3 [00:00<?, ?it/s]

y_train shape for P ontology: (82404, 16858) 		 Number of unique P terms: 16858


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
Training models:  33%|███▎      | 1/3 [2:29:37<4:59:15, 8977.86s/it]

Model for P trained successfully.
y_train shape for C ontology: (82404, 2651) 		 Number of unique C terms: 2651


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Model for C trained successfully.
y_train shape for F ontology: (82404, 6616) 		 Number of unique F terms: 6616


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Model for F trained successfully.


## Step 5: Inference and Submission

In [14]:
test_sequences_df = load_fasta_to_dataframe(TEST_SEQ, is_train=False)
test_entry_ids = test_sequences_df['EntryID'].tolist()
test_sequences_dict = test_sequences_df.set_index('EntryID')['sequence'].to_dict()

BATCH_SIZE = 5000  # avoid overload
submission_list = []

for i in tqdm(range(0, len(test_entry_ids), BATCH_SIZE), desc="Predicting on Test Set"):
    batch_entry_ids = test_entry_ids[i:i + BATCH_SIZE]
    batch_seqs = [test_sequences_dict[entry_id] for entry_id in batch_entry_ids]
    X_batch = np.array([extract_sequence_features(seq) for seq in batch_seqs])
    
    for aspect, model in models.items():
        mlb = mlb_dict[aspect]
        y_pred_proba = model.predict_proba(X_batch)
        
        for j, entry_id in enumerate(batch_entry_ids):
            probs = y_pred_proba[j]
            candidate_indices = np.where(probs > 0.02)[0]
            
            for idx in candidate_indices:
                submission_list.append((entry_id, mlb.classes_[idx], round(probs[idx], 3)))

print(f"Generated {len(submission_list):,} total predictions.")

Predicting on Test Set: 100%|██████████| 45/45 [16:19<00:00, 21.77s/it]

Generated 4,886,535 total predictions.


In [15]:
submission_df = pd.DataFrame(submission_list, columns=['Protein Id', 'GO Term Id', 'Prediction'])

print("Applying 1500 prediction limit per protein...")
submission_df = submission_df.sort_values(by=['Protein Id', 'Prediction'], ascending=[True, False])
final_submission_df = submission_df.groupby('Protein Id').head(1500).reset_index(drop=True)
final_submission_df.to_csv('submission.tsv', sep='\t', index=False, header=False)

print("\nSubmission file 'submission.tsv' created successfully.")
print(f"Total predictions in final submission: {len(final_submission_df):,}")
print("Submission DataFrame Head:")
display(final_submission_df.head())

Applying 1500 prediction limit per protein...

Submission file 'submission.tsv' created successfully.
Total predictions in final submission: 4,768,132
Submission DataFrame Head:


,Protein Id,GO Term Id,Prediction
0,A0A017SE81,GO:0005515,0.230
1,A0A017SE81,GO:0005886,0.197
2,A0A017SE81,GO:0005739,0.132
3,A0A017SE81,GO:0005829,0.086
4,A0A017SE81,GO:0005634,0.066
